RAW (MOABB) to CSV

This code convert the data sets from RAW format to CSV format using MOABB.

It has been specifically conceived for BCI data.

This script is for EPFLP300-1 (Telephone run 1)


In [ ]:
#import essential libraries

import numpy as np
import pandas as pd
import moabb.datasets

In [ ]:
#Load Database
m_dataset = moabb.datasets.EPFLP300()
m_data = m_dataset.get_data()


In [ ]:
#See all canal names (EEG, misc, stim...)
raw = m_data[1]['0']['0']
print("Canal list :", raw.ch_names)

In [ ]:
#Know what index is stim channel (we will need it later for the CSV to NY conversion)
stim_channel_name = 'STI'
stim_idx = raw.ch_names.index(stim_channel_name)
print(f"Canal index {stim_channel_name} is : {stim_idx}")

In [ ]:
#count stim data unique values (1 non target, 2 = target with a ratio needed of 5 to 1)
stim_data = raw.get_data(picks=stim_idx)
print(stim_data.shape)
unique_vals, counts = np.unique(stim_data, return_counts=True)

for val, count in zip(unique_vals, counts):
    print(f"Value : {val}, Occurences count : {count}")

In [ ]:
# Transpose to invert columns/lines
data = raw.get_data()
dataT = data.T

In [ ]:
# creating timestamps and header
n_times, n_channels = dataT.shape
timestamps = np.arange(n_times, dtype=int)
data_with_timestamp = np.column_stack((timestamps, dataT))
header = [""] + [str(i) for i in range(n_channels)]

# Removing decimals from timestamps
df = pd.DataFrame(data_with_timestamp, columns=header)
df[""] = df[""].astype(int)

In [ ]:
# Test to check csv file for sub 1 sess 1 run 1
df.to_csv("data.csv", index=False)

In [ ]:
# LOOP TO CREATE ALL FILES OF RUN 1 FROM DATABASE

subject_list = [1, 2, 3, 4, 6, 7, 8, 9]
num_sessions = 4

for subject in subject_list:
    for session_idx in range(num_sessions):
        raw = m_data[subject][str(session_idx)]['0']
        data = raw.get_data()
        dataT = data.T
        n_times, n_channels = dataT.shape
        timestamps = np.arange(n_times, dtype=int)
        datacsv = np.column_stack((timestamps, dataT))
        header = [""] + [str(i) for i in range(n_channels)]
        df = pd.DataFrame(datacsv, columns=header)
        df[""] = df[""].astype(int)
        subject_str = f"{subject:02d}"
        session_str = f"{session_idx+1:02d}"
        filename = f"subject_{subject_str}_session_{session_str}.csv"
        df.to_csv(filename, index=False)

        print(f"Saved file : {filename}")